In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
data = pd.read_stata('data_Deming_2008_0217.dta')
data[data < 0] = np.nan # replace negative values with NaN
data

,ChildID,AFQT_Pct81,AFQT_Pct81_REV,AgePreg94,AgePreg96,AgePreg98,Age_1stHS88,Age_1stHS90,Age_1stHS92,Age_1stHS94,...,YA_LD104,YA_LastInterview,YA_NumKids,YA_Res94,YA_Res96,YA_Res98,YA_Res100,YA_Res102,YA_Res104,hhID
0,201.0,12.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,202.0,12.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,301.0,51.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2002.0,0.0,NaN,11.0,19.0,NaN,19.0,NaN,3.0
3,302.0,51.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2002.0,0.0,NaN,NaN,19.0,11.0,11.0,NaN,3.0
4,303.0,51.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2002.0,0.0,NaN,NaN,NaN,NaN,19.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11465,1267201.0,64.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12672.0
11466,1267202.0,64.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12672.0
11467,1267301.0,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12673.0
11468,1267302.0,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12673.0


In [3]:
# Eligibility

# age
survey_years = list(range(86, 106, 2))
for year in survey_years:
    data[f'cleaned_age_mo_{year}'] = data[f'Age_Mo{year}']
    data[f'cleaned_age_mo_{year}'].fillna(data[f'PPVTAge{year}'], inplace=True)
for idx, year in enumerate(survey_years):
    if idx != 0:
        mask = data[f'cleaned_age_mo_{year}'].isna() & data[f'cleaned_age_mo_{year-2}'].notna()
        data.loc[mask, f'cleaned_age_mo_{year}'] = data.loc[mask, f'cleaned_age_mo_{year-2}'] + 24
    if idx != len(survey_years) - 1:
        mask = data[f'cleaned_age_mo_{year}'].isna() & data[f'cleaned_age_mo_{year+2}'].notna() & data[f'cleaned_age_mo_{year+2}'] >= 25
        data.loc[mask, f'cleaned_age_mo_{year}'] = data.loc[mask, f'cleaned_age_mo_{year+2}'] - 24
for year in survey_years:
    data[f'cleaned_age_yr_{year}'] = data[f'cleaned_age_mo_{year}'] // 12

# each household must have at least two eligible children
grouped_data = data.groupby('MotherID')
for year in [90]:
    data[f'num_eligible_children_{year}'] = grouped_data[f'cleaned_age_mo_{year}'].transform(lambda x: x[(x >= 54) & (x.notna())].count()) # 4 yr 6 months
    eligibility_mask = (data[f'num_eligible_children_{year}'] > 1) & (data[f'num_eligible_children_{year}'].notna())
    data.loc[eligibility_mask, f'eligibility_{year}'] = 1
    data.loc[~eligibility_mask, f'eligibility_{year}'] = 0

    death_mask = data[f'Res{year}'] == 8
    data.loc[death_mask, f'eligibility_{year}'] = np.nan
data['deceased'] = data['Res104'] == 8

# program participation
headstart_participation_mask = (data['Ever_HS90'] == 1) | (data['Ever_HS88'] == 1)
preschool_participation_mask = (data['Ever_Preschool90'] == 1) | (data['Ever_Preschool88'] == 1)
eligibility_mask = data['eligibility_90'] == 1
data['headstart_participation_90'] = headstart_participation_mask.apply(lambda x: x if np.isnan(x) else int(x))
data.loc[~eligibility_mask, 'headstart_participation_90'] = np.nan
data['preschool_participation_90'] = (preschool_participation_mask & ~headstart_participation_mask).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[~eligibility_mask, 'preschool_participation_90'] = np.nan
data['no_preschool_participation_90'] = (~preschool_participation_mask & ~headstart_participation_mask).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[~eligibility_mask, 'no_preschool_participation_90'] = np.nan

# within family treatment difference
grouped_data = data.groupby('MotherID')
year = 90

hs_indicator = grouped_data[f'headstart_participation_{year}'].transform(lambda x: 1 if x.sum() > 0 else 0)
pr_indicator = grouped_data[f'preschool_participation_{year}'].transform(lambda x: 1 if x.sum() > 0 else 0)
no_indicator = grouped_data[f'no_preschool_participation_{year}'].transform(lambda x: 1 if x.sum() > 0 else 0)

data[f'eligibility_siblingdifferenttreatment_{year}'] = ((hs_indicator + pr_indicator + no_indicator) >= 2) & (data[f'eligibility_{year}'] == 1)
data[f'headstart_fixedeffect_indicator_{year}'] = data[f'headstart_participation_{year}']
data[f'preschool_fixedeffect_indicator_{year}'] = data[f'preschool_participation_{year}']
data[f'no_preschool_fixedeffect_indicator_{year}'] = data[f'no_preschool_participation_{year}']
data.loc[~data[f'eligibility_siblingdifferenttreatment_{year}'], f'headstart_fixedeffect_indicator_{year}'] = np.nan
data.loc[~data[f'eligibility_siblingdifferenttreatment_{year}'], f'preschool_fixedeffect_indicator_{year}'] = np.nan
data.loc[~data[f'eligibility_siblingdifferenttreatment_{year}'], f'no_preschool_fixedeffect_indicator_{year}'] = np.nan
data[f'eligibility_siblingdifferenttreatment_{year}'] = data[f'eligibility_siblingdifferenttreatment_{year}'].apply(lambda x: x if np.isnan(x) else int(x))


data[f"program_participation_type_{year}"] = data[f'headstart_participation_{year}'] + 2 * data[f'preschool_participation_{year}'] + 3 * data[f'no_preschool_participation_{year}']
data.loc[data[f'eligibility_{year}'] == 0, f"program_participation_type_{year}"] = np.nan

data[f"program_participation_fixedeffect_type_{year}"] = data[f'headstart_fixedeffect_indicator_{year}'] + 2 * data[f'preschool_fixedeffect_indicator_{year}'] + 3 * data[f'no_preschool_fixedeffect_indicator_{year}']
data.loc[data[f'eligibility_siblingdifferenttreatment_{year}'] == 0, f"program_participation_fixedeffect_type_{year}"] = np.nan

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/2578242216.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'cleaned_age_mo_{year}'] = data[f'Age_Mo{year}']
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/2578242216.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original 

In [4]:
# Covariates

# race
data['Hispanic'] = (data['Race_Child'] == 1).apply(lambda x: x if np.isnan(x) else int(x))
data['Black'] = (data['Race_Child'] == 2).apply(lambda x: x if np.isnan(x) else int(x))
data['White'] = (data['Race_Child'] == 3).apply(lambda x: x if np.isnan(x) else int(x))
data['NonBlack'] = ((data['Race_Child'] != 2) & (data['Race_Child'].notna())).apply(lambda x: x if np.isnan(x) else int(x))

# family income, adjust to 2004 dollars
data['NetFamInc78'] *= 2.82
data['NetFamInc79'] *= 2.54
data['NetFamInc80'] *= 2.24
data['NetFamInc81'] *= 2.03
data['NetFamInc82'] *= 1.90
data['NetFamInc83'] *= 1.85
data['NetFamInc84'] *= 1.78
data['NetFamInc85'] *= 1.71
data['NetFamInc86'] *= 1.68
data['NetFamInc87'] *= 1.62
data['NetFamInc88'] *= 1.55
data['NetFamInc89'] *= 1.48
data['NetFamInc90'] *= 1.41
data['NetFamInc91'] *= 1.35
data['NetFamInc92'] *= 1.31
data['NetFamInc93'] *= 1.27
data['NetFamInc95'] *= 1.21
data['NetFamInc97'] *= 1.15
data['NetFamInc99'] *= 1.10
data['NetFamInc101'] *= 1.04

data['permanent_family_income'] = data[['NetFamInc78', 'NetFamInc79', 'NetFamInc80', 'NetFamInc81', 'NetFamInc82', 'NetFamInc83', 'NetFamInc84',
                                        'NetFamInc85', 'NetFamInc86', 'NetFamInc87', 'NetFamInc88', 'NetFamInc89', 'NetFamInc90', 'NetFamInc91',
                                        'NetFamInc92', 'NetFamInc93', 'NetFamInc95', 'NetFamInc97', 'NetFamInc99', 'NetFamInc101']].mean(axis=1)
data['log_permanent_family_income'] = np.log(data['permanent_family_income'])
data['permanent_family_income_std'] = (data['permanent_family_income'] - data['permanent_family_income'].mean()) / data['permanent_family_income'].std() # standardize

# mother's education
high_grade_moth_cols = [col for col in data.columns if col.startswith('HighGrade_Moth')]
for col in high_grade_moth_cols:
    data[col].replace(95, np.nan, inplace=True)
mother_edu = data[high_grade_moth_cols].max(axis=1)

data['mother_dropout'] = (mother_edu < 12).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[mother_edu.isna(), 'mother_dropout'] = np.nan
data['mother_highschool'] = (mother_edu == 12).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[mother_edu.isna(), 'mother_highschool'] = np.nan
data['mother_somecollege'] = (mother_edu >= 13).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[mother_edu.isna(), 'mother_somecollege'] = np.nan

# maternal AFQT score
data['age_adjusted_AFQT'] = data['AFQT_Pct81_REV']
age_adjustment_factors = {
    14: 35.60881 / 28.79544,
    15: 35.60881 / 32.86273,
    16: 35.60881 / 32.86273,
    17: 35.60881 / 36.3544,
    18: 35.60881 / 33.45777,
    19: 35.60881 / 36.84,
    20: 35.60881 / 41.84536,
    21: 35.60881 / 40.95177,
    22: 35.60881 / 42.82069
}
for age, factor in age_adjustment_factors.items():
    data.loc[data['Age_Mom79'] == age, 'age_adjusted_AFQT'] *= factor
data['age_adjusted_AFQT_std'] = (data['age_adjusted_AFQT'] - data['age_adjusted_AFQT'].mean()) / data['age_adjusted_AFQT'].std()

# Impute missing values for AgeAFQT_std
data['impAFQT_std'] = data['age_adjusted_AFQT_std']
conditional_means = data.groupby(['Black', 'Hispanic', 'Age_Moth_Birth'])['age_adjusted_AFQT_std'].transform('mean')
data['impAFQT_std'] = data['impAFQT_std'].fillna(conditional_means)

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1616015331.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Hispanic'] = (data['Race_Child'] == 1).apply(lambda x: x if np.isnan(x) else int(x))
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1616015331.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Black'] = (data['Race_Child'] == 2).apply(lambda x: x if np.isnan(x) else int(x))
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1616015331.py:6: PerformanceWarnin

In [5]:
# Table 1
covariates_to_list = ["permanent_family_income", "mother_dropout", "mother_somecollege", "age_adjusted_AFQT_std", "HighGrade_GMom79"]
column_names = ["White/Hispanic, Head Start", "White/Hispanic, Preschool", "White/Hispanic, None",
                "Black, Head Start", "Black, Preschool", "Black, None",
                "Head start—none diff. (in SD units), White/Hispanic", "Head start—none diff. (in SD units), Black"]
row_names = []
for covariate in covariates_to_list:
    row_names.append(covariate)
    row_names.append(covariate + " std")
    row_names.append(covariate + "Fixed effects subsample")
    row_names.append(covariate + "Fixed effects subsample std")
row_names.append("Sample size")
row_names.append("Fixed effects subsample size")
table1 = pd.DataFrame(index=row_names, columns=column_names)
for idx, covariate in enumerate(covariates_to_list):
    mask = (data['NonBlack'] == 1) & (data['program_participation_type_90'] == 1)
    table1.loc[covariate, "White/Hispanic, Head Start"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "White/Hispanic, Head Start"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "White/Hispanic, Head Start"] = mask.sum()
    mask = (data['NonBlack'] == 1) & (data['program_participation_type_90'] == 2)
    table1.loc[covariate, "White/Hispanic, Preschool"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "White/Hispanic, Preschool"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "White/Hispanic, Preschool"] = mask.sum()
    mask = (data['NonBlack'] == 1) & (data['program_participation_type_90'] == 3)
    table1.loc[covariate, "White/Hispanic, None"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "White/Hispanic, None"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "White/Hispanic, None"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_type_90'] == 1)
    table1.loc[covariate, "Black, Head Start"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "Black, Head Start"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "Black, Head Start"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_type_90'] == 2)
    table1.loc[covariate, "Black, Preschool"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "Black, Preschool"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "Black, Preschool"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_type_90'] == 3)
    table1.loc[covariate, "Black, None"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + " std", "Black, None"] = data.loc[mask, covariate].std()
    table1.loc["Sample size", "Black, None"] = mask.sum()

    mask_all = (data['NonBlack'] == 1) & (data['program_participation_type_90'].notna())
    mask_hs = (data['NonBlack'] == 1) & (data['program_participation_type_90'] == 1)
    mask_none = (data['NonBlack'] == 1) & (data['program_participation_type_90'] == 3)
    table1.loc[covariate, "Head start—none diff. (in SD units), White/Hispanic"] = (data.loc[mask_hs, covariate].mean() - data.loc[mask_none, covariate].mean()) / data.loc[mask_all, covariate].std()
    table1.loc[covariate + " std", "Head start—none diff. (in SD units), White/Hispanic"] = np.nan

    mask_all = (data['Black'] == 1) & (data['program_participation_type_90'].notna())
    mask_hs = (data['Black'] == 1) & (data['program_participation_type_90'] == 1)
    mask_none = (data['Black'] == 1) & (data['program_participation_type_90'] == 3)
    table1.loc[covariate, "Head start—none diff. (in SD units), Black"] = (data.loc[mask_hs, covariate].mean() - data.loc[mask_none, covariate].mean()) / data.loc[mask_all, covariate].std()
    table1.loc[covariate + " std", "Head start—none diff. (in SD units), Black"] = np.nan


    mask = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'] == 1)
    table1.loc[covariate + "Fixed effects subsample", "White/Hispanic, Head Start"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "White/Hispanic, Head Start"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "White/Hispanic, Head Start"] = mask.sum()
    mask = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'] == 2)
    table1.loc[covariate + "Fixed effects subsample", "White/Hispanic, Preschool"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "White/Hispanic, Preschool"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "White/Hispanic, Preschool"] = mask.sum()
    mask = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'] == 3)
    table1.loc[covariate + "Fixed effects subsample", "White/Hispanic, None"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "White/Hispanic, None"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "White/Hispanic, None"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'] == 1)
    table1.loc[covariate + "Fixed effects subsample", "Black, Head Start"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "Black, Head Start"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "Black, Head Start"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'] == 2)
    table1.loc[covariate + "Fixed effects subsample", "Black, Preschool"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "Black, Preschool"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "Black, Preschool"] = mask.sum()
    mask = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'] == 3)
    table1.loc[covariate + "Fixed effects subsample", "Black, None"] = data.loc[mask, covariate].mean()
    table1.loc[covariate + "Fixed effects subsample std", "Black, None"] = data.loc[mask, covariate].std()
    table1.loc["Fixed effects subsample size", "Black, None"] = mask.sum()

    mask_all = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'].notna())
    mask_hs = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'] == 1)
    mask_none = (data['NonBlack'] == 1) & (data['program_participation_fixedeffect_type_90'] == 3)
    table1.loc[covariate + "Fixed effects subsample", "Head start—none diff. (in SD units), White/Hispanic"] = (data.loc[mask_hs, covariate].mean() - data.loc[mask_none, covariate].mean()) / data.loc[mask_all, covariate].std()
    table1.loc[covariate + "Fixed effects subsample std", "Head start—none diff. (in SD units), White/Hispanic"] = np.nan

    mask_all = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'].notna())
    mask_hs = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'] == 1)
    mask_none = (data['Black'] == 1) & (data['program_participation_fixedeffect_type_90'] == 3)
    table1.loc[covariate + "Fixed effects subsample", "Head start—none diff. (in SD units), Black"] = (data.loc[mask_hs, covariate].mean() - data.loc[mask_none, covariate].mean()) / data.loc[mask_all, covariate].std()
    table1.loc[covariate + "Fixed effects subsample std", "Head start—none diff. (in SD units), Black"] = np.nan

table1.to_csv("table1.csv")
table1

,"White/Hispanic, Head Start","White/Hispanic, Preschool","White/Hispanic, None","Black, Head Start","Black, Preschool","Black, None","Head start—none diff. (in SD units), White/Hispanic","Head start—none diff. (in SD units), Black"
permanent_family_income,26388.322266,50042.378906,35153.972656,22788.820312,32404.78125,25210.658203,-0.28597,-0.114621
permanent_family_income std,19458.916016,45940.242188,23423.578125,14835.428711,26156.855469,21755.886719,NaN,NaN
permanent_family_incomeFixed effects subsample,26575.185547,45532.839844,36482.328125,23876.337891,30637.169922,23698.416016,-0.397885,0.008844
permanent_family_incomeFixed effects subsample std,21131.683594,25011.263672,24515.021484,16325.375977,26975.996094,18691.693359,NaN,NaN
mother_dropout,0.522565,0.197659,0.437905,0.362222,0.226054,0.399354,0.173281,-0.077261
mother_dropout std,0.500085,0.398493,0.496244,0.481178,0.419078,0.490029,NaN,NaN
mother_dropoutFixed effects subsample,0.498403,0.214286,0.378587,0.400651,0.256281,0.382524,0.250569,0.03767
mother_dropoutFixed effects subsample std,0.500798,0.410745,0.485303,0.490831,0.43768,0.486476,NaN,NaN
mother_somecollege,0.218527,0.390117,0.225209,0.288889,0.482759,0.286329,-0.015187,0.005493
mother_somecollege std,0.413739,0.488094,0.417816,0.453751,0.500663,0.452289,NaN,NaN


In [6]:
# More covariates

# live in the same household as mother, 0-3 years old
for year in range(79, 92):
    data[f'live_with_mother_{year}'] = np.nan
    mask = data[f'Res{year}'] == 1
    data.loc[mask, f'live_with_mother_{year}'] = 1
    mask = (data[f'Res{year}'] != 1) & (data[f'Res{year}'].notna())
    data.loc[mask, f'live_with_mother_{year}'] = 0

data[f'live_with_mother_{92}'] = data[f'live_with_mother_{91}']
data[f'live_with_mother_{93}'] = data[f'live_with_mother_{91}']
data[f'live_with_mother_{78}'] = data[f'live_with_mother_{79}']
data[f'live_with_mother_{77}'] = data[f'live_with_mother_{79}']
data[f'live_with_mother_{76}'] = data[f'live_with_mother_{79}']

data["reside_with_mother_0to3"] = np.nan
for year in range(76, 91):
    mask = (104 - data[f'cleaned_age_yr_104'] == year)
    live_mask = (data[f'live_with_mother_{year}'] == 1) & (data[f'live_with_mother_{year+1}'] <= 1) & (data[f'live_with_mother_{year+2}'] <= 1) & (data[f'live_with_mother_{year+3}'] <= 1)
    missing_mask = (data[f'live_with_mother_{year}'].isna()) & (data[f'live_with_mother_{year+1}'].isna()) & (data[f'live_with_mother_{year+2}'].isna()) & (data[f'live_with_mother_{year+3}'].isna())
    data.loc[mask & live_mask, "reside_with_mother_0to3"] = 1
    data.loc[mask & ~live_mask, "reside_with_mother_0to3"] = 0
    data.loc[mask & missing_mask, "reside_with_mother_0to3"] = np.nan

for year in range(76, 94):
    del data[f'live_with_mother_{year}']

# Preexisting health conditions prior to age 5
data["Ear86"] = data["Ear88"]
data["Blood86"] = data["Blood88"]
data["Epilepsy86"] = data["Epilepsy88"]
health_conditions = ["Brain", "Hyper", "Asthma", "Resp", "Speech", "Deaf", "Blind", "Disturb", "Allergy", "Crippled", "Retard", "Heart", "Nerve", "Ear", "Blood", "Epilepsy", "OtherLim"]
for condition in health_conditions:
    for year in range(86, 92, 2):
        data[f'temp{condition}{year}'] = np.nan
        mask = data[f'{condition}{year}'] > 0
        data.loc[mask, f'temp{condition}{year}'] = 1
    data[condition] = data[[f'temp{condition}{year}' for year in range(86, 92, 2)]].max(axis=1)
    data[f'{condition}_before'] = np.nan
    for year in range(86, 92, 2):
        mask = (data[f'temp{condition}{year}'] == 1) & (data[f'cleaned_age_mo_{year}'] < 60)
        data.loc[mask, f'{condition}_before'] = 1
    data.drop(columns=[f'temp{condition}{year}' for year in range(86, 92, 2)], inplace=True)

data['prexisting_health_conditions'] = data[[f'{condition}_before' for condition in health_conditions]].max(axis=1)
data.loc[data["reside_with_mother_0to3"].isna(), 'prexisting_health_conditions'] = np.nan
data.loc[(~data["reside_with_mother_0to3"].isna()) & data['prexisting_health_conditions'].isna(), 'prexisting_health_conditions'] = 0 # TODO: ad hoc


# low birth weight
data['low_birth_weight'] = (data['BirthWeight'] < 88).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[data['BirthWeight'].isna(), 'low_birth_weight'] = np.nan

data['very_low_birth_weight'] = (data['BirthWeight'] < 53).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[data['BirthWeight'].isna(), 'very_low_birth_weight'] = np.nan

# attrition: if respondent disappears from sample before age 19
data['Attrit'] = np.nan
data.loc[data['YA_LastInterview'] == 2004, 'Attrit'] = 0
data.loc[(data['YA_LastInterview'] != 2004) & data['YA_LastInterview'].notna(), 'Attrit'] = 1
for year in [94, 96, 98, 100, 102]:
    data.loc[(data['Attrit'] == 1) & (data['YA_LastInterview'] == 1900 + year) & (data[f'cleaned_age_yr_{year}'] >= 19), 'Attrit'] = 0

# some dummy for estimation
data["Sample90"] = np.nan
mask = (data['eligibility_siblingdifferenttreatment_90'] == 1) & (data['Attrit'] == 0) &\
      (((data['SampleID'] != 12) & (data['cleaned_age_yr_104'] >= 19)) | ((data['DOB_Yr_Child']==1985) & (data['DOB_Mo_Child']<8)))
data.loc[mask, "Sample90"] = 1


# Generate the 'HS_3' variable
data['Three'] = ((data['Age_1stHS88'] <= 3) | (data['Age_1stHS90'] <= 3)).apply(lambda x: x if np.isnan(x) else int(x))
data['NotThree'] = ((data['Age_1stHS88'] > 3) & (data['Age_1stHS88'].notna()) | (data['Age_1stHS90'] > 3) & (data['Age_1stHS90'].notna())).apply(lambda x: x if np.isnan(x) else int(x))
data['HS_3'] = np.nan
data.loc[data['Three'] == 1, 'HS_3'] = 1
data.loc[data['NotThree'] == 1, 'HS_3'] = 2
data['HS_3'].fillna(0, inplace=True)
# Separate kids that are in Head Start at age 3 from later
data['program_participation_fixedeffect_type_90_before3'] = data['program_participation_fixedeffect_type_90']
data.loc[(data['program_participation_fixedeffect_type_90'] == 1) & (data['HS_3'] == 1), 'program_participation_fixedeffect_type_90_before3'] = 0
# so 0 is HS before age 3, 1 is HS after age 3, 2 is other pre-school program, 3 is no pre-school program

# Log Income Ages 0-3, Log Income at Age 3*
data['income_0to3'] = np.nan
for age in range(14, 30):
    temp_col = f'temp{age}'
    if age == 14:
        data[temp_col] = data['NetFamInc90']
    elif age == 15:
        data[temp_col] = data[['NetFamInc89', 'NetFamInc90']].mean(axis=1)
    elif age == 16:
        data[temp_col] = data[['NetFamInc88', 'NetFamInc89', 'NetFamInc90']].mean(axis=1)
    elif age == 17:
        data[temp_col] = data[['NetFamInc87', 'NetFamInc88', 'NetFamInc89', 'NetFamInc90']].mean(axis=1)
    elif age == 18:
        data[temp_col] = data[['NetFamInc86', 'NetFamInc87', 'NetFamInc88', 'NetFamInc89']].mean(axis=1)
    elif age == 19:
        data[temp_col] = data[['NetFamInc85', 'NetFamInc86', 'NetFamInc87', 'NetFamInc88']].mean(axis=1)
    elif age == 20:
        data[temp_col] = data[['NetFamInc84', 'NetFamInc85', 'NetFamInc86', 'NetFamInc87']].mean(axis=1)
    elif age == 21:
        data[temp_col] = data[['NetFamInc83', 'NetFamInc84', 'NetFamInc85', 'NetFamInc86']].mean(axis=1)
    elif age == 22:
        data[temp_col] = data[['NetFamInc82', 'NetFamInc83', 'NetFamInc84', 'NetFamInc85']].mean(axis=1)
    elif age == 23:
        data[temp_col] = data[['NetFamInc81', 'NetFamInc82', 'NetFamInc83', 'NetFamInc84']].mean(axis=1)
    elif age == 24:
        data[temp_col] = data[['NetFamInc80', 'NetFamInc81', 'NetFamInc82', 'NetFamInc83']].mean(axis=1)
    elif age == 25:
        data[temp_col] = data[['NetFamInc79', 'NetFamInc80', 'NetFamInc81', 'NetFamInc82']].mean(axis=1)
    elif age == 26:
        data[temp_col] = data[['NetFamInc79', 'NetFamInc80', 'NetFamInc81']].mean(axis=1)
    elif age == 27:
        data[temp_col] = data[['NetFamInc78', 'NetFamInc79', 'NetFamInc80']].mean(axis=1)
    elif age == 28:
        data[temp_col] = data[['NetFamInc78', 'NetFamInc79']].mean(axis=1)
    elif age == 29:
        data[temp_col] = data['NetFamInc78']
    data.loc[data['cleaned_age_yr_104'] == age, 'income_0to3'] = data[temp_col]

data.drop(columns=[f'temp{age}' for age in range(14, 30)], inplace=True)
data['log_income_0to3'] = np.log(data['income_0to3'])
data['log_income_0to3'].replace(-np.inf, np.nan, inplace=True)

data['income_at_3'] = np.nan
for age in range(14, 30):
    data.loc[data['cleaned_age_yr_104'] == age, 'income_at_3'] = data[f'NetFamInc{104 - age + 3}']
data['log_income_at_3'] = np.log(data['income_at_3'])
data['log_income_at_3'].replace(-np.inf, np.nan, inplace=True)


# first born
data['first_born'] = (data['BirthOrder']==1).apply(lambda x: x if np.isnan(x) else int(x))

# male
data['male'] = (data['Sex_Child']==1).apply(lambda x: x if np.isnan(x) else int(x))

# PPVT score at age 3
data['PPVTat3'] = np.nan
mask_86 = (data['PPVTAge86'] >= 36) & (data['PPVTAge86'] < 47)
data.loc[mask_86, 'PPVTat3'] = data['PPVT_Raw86']
mask_88 = (data['PPVTAge88'] >= 36) & (data['PPVTAge88'] < 47) & data['PPVTat3'].isna()
data.loc[mask_88, 'PPVTat3'] = data['PPVT_Raw88']
mask_90 = (data['PPVTAge90'] >= 36) & (data['PPVTAge90'] < 47) & data['PPVTat3'].isna()
data.loc[mask_90, 'PPVTat3'] = data['PPVT_Raw90']

# HOME score
data['HOME_Pct_0to3'] = np.nan
mask_16_19 = (data['cleaned_age_yr_104'] <= 19) & (data['cleaned_age_yr_104'] >= 16)
mask_14_15 = (data['cleaned_age_yr_104'] <= 15) & (data['cleaned_age_yr_104'] >= 14)
mask_20_21 = (data['cleaned_age_yr_104'] >= 20) & (data['cleaned_age_yr_104'] <= 21)
data.loc[mask_16_19, 'HOME_Pct_0to3'] = data[['HOME_Pct86', 'HOME_Pct88']].mean(axis=1)
data.loc[mask_14_15, 'HOME_Pct_0to3'] = data[['HOME_Pct88', 'HOME_Pct90']].mean(axis=1)
data.loc[mask_20_21, 'HOME_Pct_0to3'] = data['HOME_Pct86']

# Mom work
data['Moth_HrsWorked_BefBirth'] = data[[f'Moth_HrsWorked_{qtr}_Qtr_Before' for qtr in range(1, 5)]].mean(axis=1) / 13
data['Moth_HrsWorked_0to3'] = data[[f'Moth_HrsWorked_{qtr}_Qtr' for qtr in range(1, 13)]].mean(axis=1)
data['Moth_HrsWorked_Avg_0to3'] = data[[f'Moth_HrsWorked_{qtr}_Avg' for qtr in range(1, 13)]].mean(axis=1)
data['Moth_HrsWorked_0to1'] = data[[f'Moth_HrsWorked_{qtr}_Avg' for qtr in range(1, 5)]].mean(axis=1)

# Father present 0-3
data['Father_HH_0to3'] = np.nan
data.loc[data['cleaned_age_yr_104'] == 14, 'Father_HH_0to3'] = data[['Father_HH90', 'Father_HH92', 'Father_HH93']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 15, 'Father_HH_0to3'] = data[['Father_HH89', 'Father_HH90']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 16, 'Father_HH_0to3'] = data[['Father_HH88', 'Father_HH89', 'Father_HH90']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 17, 'Father_HH_0to3'] = data[['Father_HH87', 'Father_HH88', 'Father_HH89', 'Father_HH90']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 18, 'Father_HH_0to3'] = data[['Father_HH86', 'Father_HH87', 'Father_HH88', 'Father_HH89']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 19, 'Father_HH_0to3'] = data[['Father_HH85', 'Father_HH86', 'Father_HH87', 'Father_HH88']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 20, 'Father_HH_0to3'] = data[['Father_HH84', 'Father_HH85', 'Father_HH86', 'Father_HH87']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 21, 'Father_HH_0to3'] = data[['Father_HH84', 'Father_HH85', 'Father_HH86']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 22, 'Father_HH_0to3'] = data[['Father_HH84', 'Father_HH85']].mean(axis=1)
data.loc[data['cleaned_age_yr_104'] == 23, 'Father_HH_0to3'] = data['Father_HH84']

# Grandmother present 0-3
for year in range(79, 91):
    data[f'GMom{year}'] = np.nan
    mask = data[f'Grandmother{year}'] == 1
    data.loc[mask, f'GMom{year}'] = 1
    mask = (data[f'Grandmother{year}'] != 1) & (data[f'Grandmother{year}'].notna())
    data.loc[mask, f'GMom{year}'] = 0

data['GMom_0to3'] = np.nan
for age, cols in zip(range(14, 29), [
    ['GMom90'],
    ['GMom89', 'GMom90'],
    ['GMom88', 'GMom89', 'GMom90'],
    ['GMom87', 'GMom88', 'GMom89', 'GMom90'],
    ['GMom86', 'GMom87', 'GMom88', 'GMom89'],
    ['GMom85', 'GMom86', 'GMom87', 'GMom88'],
    ['GMom84', 'GMom85', 'GMom86', 'GMom87'],
    ['GMom83', 'GMom84', 'GMom85', 'GMom86'],
    ['GMom82', 'GMom83', 'GMom84', 'GMom85'],
    ['GMom81', 'GMom82', 'GMom83', 'GMom84'],
    ['GMom80', 'GMom81', 'GMom82', 'GMom83'],
    ['GMom79', 'GMom80', 'GMom81', 'GMom82'],
    ['GMom79', 'GMom80', 'GMom81'],
    ['GMom79', 'GMom80'],
    ['GMom79']
]):
    data.loc[data['cleaned_age_yr_104'] == age, 'GMom_0to3'] = data[cols].mean(axis=1)

data.drop(columns=[f'GMom{year}' for year in range(79, 91)], inplace=True)


# Rename columns
data.rename(columns={
    'ChildCare_1stYr': 'ChildCare_1_Yr',
    'ChildCare_Type_1stYr': 'ChildCare_Type_1_Yr',
    'ChildCare_2ndYr': 'ChildCare_2_Yr',
    'ChildCare_Type_2ndYr': 'ChildCare_Type_2_Yr',
    'ChildCare_3rdYr': 'ChildCare_3_Yr',
    'ChildCare_Type_3rdYr': 'ChildCare_Type_3_Yr'
}, inplace=True)
for y in range(1, 4):
    data[f'RelCare_{y}_Yr'] = np.where((data[f'ChildCare_Type_{y}_Yr'].notna()) & (data[f'ChildCare_Type_{y}_Yr'] <= 10), 1, np.nan)
    data[f'RelCare_{y}_Yr'] = np.where((data[f'ChildCare_{y}_Yr'].notna()) & (data[f'RelCare_{y}_Yr'] != 1), 0, data[f'RelCare_{y}_Yr'])
    
    data[f'NonRelCare_{y}_Yr'] = np.where((data[f'ChildCare_Type_{y}_Yr'].notna()) & (data[f'ChildCare_Type_{y}_Yr'] > 10), 1, np.nan)
    data[f'NonRelCare_{y}_Yr'] = np.where((data[f'ChildCare_{y}_Yr'].notna()) & (data[f'NonRelCare_{y}_Yr'] != 1), 0, data[f'NonRelCare_{y}_Yr'])
    
    data[f'MomCare_{y}_Yr'] = np.where((data[f'RelCare_{y}_Yr'] == 0) & (data[f'NonRelCare_{y}_Yr'] == 0), 1, np.nan)
    data[f'MomCare_{y}_Yr'] = np.where((data[f'MomCare_{y}_Yr'] != 1) & (data[f'RelCare_{y}_Yr'].notna()) & (data[f'NonRelCare_{y}_Yr'].notna()), 0, data[f'MomCare_{y}_Yr'])

data['RelCare'] = data[[f'RelCare_{y}_Yr' for y in range(1, 4)]].mean(axis=1)
data['NonRelCare'] = data[[f'NonRelCare_{y}_Yr' for y in range(1, 4)]].mean(axis=1)
data['MomCare'] = data[[f'MomCare_{y}_Yr' for y in range(1, 4)]].mean(axis=1)


# Mom smoked, Mom drank, Breastfed, Doctor's visit in last 3 months, Dentist ever, Weight Change during preg
data['Alc_BefBirth'] = (data['Freq_Alc_BefBirth'] >= 3).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[data['Freq_Alc_BefBirth'].isna(), 'Alc_BefBirth'] = np.nan

for x in ['Doctor', 'Dentist']:
    data[f'{x}_temp'] = np.nan
    mask_16_19 = (data['cleaned_age_yr_104'] <= 19) & (data['cleaned_age_yr_104'] >= 16)
    mask_14_15 = (data['cleaned_age_yr_104'] <= 15) & (data['cleaned_age_yr_104'] >= 14)
    mask_20_21 = (data['cleaned_age_yr_104'] >= 20) & (data['cleaned_age_yr_104'] <= 21)
    data.loc[mask_16_19, f'{x}_temp'] = data[[f'Last_{x}86', f'Last_{x}88']].mean(axis=1)
    data.loc[mask_14_15, f'{x}_temp'] = data[[f'Last_{x}88', f'Last_{x}90']].mean(axis=1)
    data.loc[mask_20_21, f'{x}_temp'] = data[f'Last_{x}86']
data['Doctor_0to3'] = np.nan
data.loc[data['Doctor_temp'] <= 2, 'Doctor_0to3'] = 1
data.loc[data['Doctor_temp'] > 2, 'Doctor_0to3'] = 0

# Dentist visits 0 to 3 years
data['Dentist_0to3'] = np.nan
data.loc[data['Dentist_temp'] < 7, 'Dentist_0to3'] = 1
data.loc[data['Dentist_temp'] == 7, 'Dentist_0to3'] = 0

# Drop temporary columns
data.drop(columns=['Doctor_temp', 'Dentist_temp'], inplace=True)

# unchanged: Moth_Smoke_BefBirth, Breastfed, Moth_WeightChange



# Illness in 1st year, Premature birth, Birthweight, Priv Health Insurance 0-3, Medicaid 0-3
data['Premature'] = (data['BornEarlyorLate'] == 1).apply(lambda x: x if np.isnan(x) else int(x))
data.loc[data['BornOnTime'].isna(), 'Premature'] = np.nan

# Priv Health Insurance 0-3, Medicaid 0-3
for x in ['Insurance', 'Medicaid']:
    data[f'{x}_0to3'] = np.nan
    mask_16_19 = (data['cleaned_age_yr_104'] <= 19) & (data['cleaned_age_yr_104'] >= 16)
    mask_14_15 = (data['cleaned_age_yr_104'] <= 15) & (data['cleaned_age_yr_104'] >= 14)
    mask_20_21 = (data['cleaned_age_yr_104'] >= 20) & (data['cleaned_age_yr_104'] <= 21)
    data.loc[mask_16_19, f'{x}_0to3'] = data[[f'{x}86', f'{x}88']].mean(axis=1)
    data.loc[mask_14_15, f'{x}_0to3'] = data[[f'{x}88', f'{x}90']].mean(axis=1)
    data.loc[mask_20_21, f'{x}_0to3'] = data[f'{x}86']

# Log Birth Weight
data['logBW'] = np.log(data['BirthWeight'])

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3303954619.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'live_with_mother_{year}'] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3303954619.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'live_with_mother_{year}'] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3303954619.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [7]:
assert np.all(data["eligibility_siblingdifferenttreatment_90"] == data["headstart_fixedeffect_indicator_90"].notna().astype(int))
mask = data['Sample90'] == 1
data.loc[mask, 'cleaned_age_yr_104'].value_counts()

cleaned_age_yr_104
21.0    183
22.0    176
20.0    171
19.0    170
23.0    156
24.0    154
25.0    123
26.0     86
27.0     75
28.0     57
29.0     41
18.0     39
30.0     14
31.0      7
32.0      2
33.0      1
Name: count, dtype: int64

In [8]:
# Table 2
covariates_to_list = ["reside_with_mother_0to3", "prexisting_health_conditions", "logBW", "log_income_0to3", "log_income_at_3", 
                      "first_born", "male", "cleaned_age_yr_104", "HOME_Pct_0to3", "Moth_HrsWorked_BefBirth", 
                      "Moth_HrsWorked_0to1", "Father_HH_0to3", "GMom_0to3", "MomCare", "RelCare", 
                      "NonRelCare", "Moth_Smoke_BefBirth", "Alc_BefBirth", "Breastfed", "Doctor_0to3", 
                      "Dentist_0to3", "Moth_WeightChange", "Illness_1stYr", "Premature", "Insurance_0to3", 
                      "Medicaid_0to3"]
mask = data['Sample90'] == 1
for covariate in covariates_to_list:
    data[covariate + "_CV"] = np.nan
    data.loc[mask, covariate + "_CV"] = (data.loc[mask, covariate] - data.loc[mask, covariate].mean()) / data.loc[mask, covariate].std()

# reverse the sign so positive means good outcomes (in regression)
varlist = ["prexisting_health_conditions", "male", "cleaned_age_yr_104", "GMom_0to3", "MomCare", "RelCare", 
           "Moth_Smoke_BefBirth", "Alc_BefBirth", "Illness_1stYr", "Premature", "Medicaid_0to3"]
for var in varlist:
    data[var + "_CV"] = -data[var + "_CV"]

data["PreTreatIndex"] = np.nan
data.loc[mask, "PreTreatIndex"] = data.loc[mask, [covariate + "_CV" for covariate in covariates_to_list]].mean(axis=1)
data.loc[mask, "PreTreatIndex"] = (data.loc[mask, "PreTreatIndex"] - data.loc[mask, "PreTreatIndex"].mean()) / data.loc[mask, "PreTreatIndex"].std()
data.drop(columns=[covariate + "_CV" for covariate in covariates_to_list], inplace=True)

covariate_names = ["reside_with_mother_0to3", "prexisting_health_conditions", "very_low_birth_weight", "logBW", "log_income_0to3", 
                      "log_income_at_3", "first_born", "male", "cleaned_age_yr_104", "HOME_Pct_0to3", 
                      "Moth_HrsWorked_BefBirth", "Moth_HrsWorked_0to1", "Father_HH_0to3", "GMom_0to3", "MomCare", 
                      "RelCare", "NonRelCare", "Moth_Smoke_BefBirth", "Alc_BefBirth", "Breastfed", 
                      "Doctor_0to3", "Dentist_0to3", "Moth_WeightChange", "Illness_1stYr", "Premature", 
                      "Insurance_0to3", "Medicaid_0to3", "PreTreatIndex"]
column_names = ["Head Start", "Other Preschool", "Control Mean", "Sample Size"]
covariate_names = ["Attrit", "PPVTat3"] + covariate_names
row_names = []
for covariate in covariate_names:
    row_names.append(covariate)
    row_names.append(covariate + " se/std")
    row_names.append(covariate + " p-value")
table2 = pd.DataFrame(index=row_names, columns=column_names)

# regression
mask = (data['Attrit'].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data["SampleID"] != 12) & ((data["cleaned_age_yr_104"] >= 19) | ((data["DOB_Yr_Child"]==1985) & (data["DOB_Mo_Child"]<8)))
model = smf.ols("Attrit ~ headstart_fixedeffect_indicator_90 + preschool_fixedeffect_indicator_90 + C(MotherID)", data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table2.loc["Attrit", "Head Start"] = model.params["headstart_fixedeffect_indicator_90"]
table2.loc["Attrit se/std", "Head Start"] = model.bse["headstart_fixedeffect_indicator_90"]
table2.loc["Attrit p-value", "Head Start"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
table2.loc["Attrit", "Other Preschool"] = model.params["preschool_fixedeffect_indicator_90"]
table2.loc["Attrit se/std", "Other Preschool"] = model.bse["preschool_fixedeffect_indicator_90"]
table2.loc["Attrit p-value", "Other Preschool"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue
table2.loc["Attrit", "Sample Size"] = mask.sum()
table2.loc["Attrit", "Control Mean"] = data.loc[mask, "Attrit"].mean()
table2.loc["Attrit se/std", "Control Mean"] = data.loc[mask, "Attrit"].std()

mask = (data['PPVTat3'].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1)
model = smf.ols("PPVTat3 ~ headstart_fixedeffect_indicator_90 + preschool_fixedeffect_indicator_90 + male + first_born + cleaned_age_mo_90 + C(MotherID)", data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table2.loc["PPVTat3", "Head Start"] = model.params["headstart_fixedeffect_indicator_90"]
table2.loc["PPVTat3 se/std", "Head Start"] = model.bse["headstart_fixedeffect_indicator_90"]
table2.loc["PPVTat3 p-value", "Head Start"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
table2.loc["PPVTat3", "Other Preschool"] = model.params["preschool_fixedeffect_indicator_90"]
table2.loc["PPVTat3 se/std", "Other Preschool"] = model.bse["preschool_fixedeffect_indicator_90"]
table2.loc["PPVTat3 p-value", "Other Preschool"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue
table2.loc["PPVTat3", "Sample Size"] = mask.sum()
table2.loc["PPVTat3", "Control Mean"] = data.loc[mask, "PPVTat3"].mean()
table2.loc["PPVTat3 se/std", "Control Mean"] = data.loc[mask, "PPVTat3"].std()

for covariate in covariate_names:
    if covariate in ["Attrit", "PPVTat3"]:
        continue
    mask = (data[covariate].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1)
    model = smf.ols(f"{covariate} ~ headstart_fixedeffect_indicator_90 + preschool_fixedeffect_indicator_90 + C(MotherID)", data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table2.loc[covariate, "Head Start"] = model.params["headstart_fixedeffect_indicator_90"]
    table2.loc[covariate + " se/std", "Head Start"] = model.bse["headstart_fixedeffect_indicator_90"]
    table2.loc[covariate + " p-value", "Head Start"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
    table2.loc[covariate, "Other Preschool"] = model.params["preschool_fixedeffect_indicator_90"]
    table2.loc[covariate + " se/std", "Other Preschool"] = model.bse["preschool_fixedeffect_indicator_90"]
    table2.loc[covariate + " p-value", "Other Preschool"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue
    table2.loc[covariate, "Sample Size"] = mask.sum()
    table2.loc[covariate, "Control Mean"] = data.loc[mask, covariate].mean()
    table2.loc[covariate + " se/std", "Control Mean"] = data.loc[mask, covariate].std()

table2.to_csv("table2.csv")
table2

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1785021321.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[covariate + "_CV"] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1785021321.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[covariate + "_CV"] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/1785021321.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

,Head Start,Other Preschool,Control Mean,Sample Size
Attrit,0.02288,-0.005789,0.04087,1517
Attrit se/std,0.018304,0.022234,0.198055,NaN
Attrit p-value,0.21131162682943538,0.7945851305256918,NaN,NaN
PPVTat3,-1.251765,-6.89092,20.724138,261
PPVTat3 se/std,21.095136,18.133818,12.941011,NaN
...,...,...,...,...
Medicaid_0to3 se/std,0.133876,0.095794,0.447503,NaN
Medicaid_0to3 p-value,0.7218903024096746,0.9460043309815034,NaN,NaN
PreTreatIndex,0.020951,0.063964,0.0,1455
PreTreatIndex se/std,0.079848,0.073031,1.0,NaN


In [9]:
covariates_to_impute = ['reside_with_mother_0to3', 'prexisting_health_conditions', 'very_low_birth_weight', 'logBW', 'log_income_0to3', 
                        'log_income_at_3', 'first_born', 'PPVTat3', 'HOME_Pct_0to3', 'Moth_HrsWorked_BefBirth', 
                        'Moth_HrsWorked_Avg_0to3', 'Moth_HrsWorked_0to1', 'Father_HH_0to3', 'GMom_0to3', 'MomCare', 
                        'RelCare', 'NonRelCare', 'Moth_Smoke_BefBirth', 'Alc_BefBirth', 'Breastfed', 
                        'Doctor_0to3', 'Dentist_0to3', 'Moth_WeightChange', 'Illness_1stYr', 'Premature', 
                        'Insurance_0to3', 'Medicaid_0to3']
main_covariates = ["male"] + covariates_to_impute


table_appendix = pd.DataFrame(index=["Test_std", "Noncog", "Sum_Adult"] + main_covariates, columns=["Mean", "Median", "Std Dev", "Head Start Mean", "Other Preschool Mean", "No Preschool Mean", "Total Sample Size"])
for covariate in main_covariates:
    mask = (data[covariate].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1)
    table_appendix.loc[covariate, "Total Sample Size"] = mask.sum()
    table_appendix.loc[covariate, "Mean"] = data.loc[mask, covariate].mean()
    table_appendix.loc[covariate, "Median"] = data.loc[mask, covariate].median()
    table_appendix.loc[covariate, "Std Dev"] = data.loc[mask, covariate].std()
    mask = (data[covariate].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1) & (data["headstart_fixedeffect_indicator_90"] == 1)
    table_appendix.loc[covariate, "Head Start Mean"] = data.loc[mask, covariate].mean()
    mask = (data[covariate].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1) & (data["preschool_fixedeffect_indicator_90"] == 1)
    table_appendix.loc[covariate, "Other Preschool Mean"] = data.loc[mask, covariate].mean()
    mask = (data[covariate].notna()) & (data["eligibility_siblingdifferenttreatment_90"] == 1) & (data['Sample90'] == 1) & (data["preschool_fixedeffect_indicator_90"] == 0) & (data["headstart_fixedeffect_indicator_90"] == 0)
    table_appendix.loc[covariate, "No Preschool Mean"] = data.loc[mask, covariate].mean()

In [10]:
# impute missing values with mean
covariates_to_impute = ['reside_with_mother_0to3', 'prexisting_health_conditions', 'very_low_birth_weight', 'logBW', 'log_income_0to3', 
                        'log_income_at_3', 'first_born', 'PPVTat3', 'HOME_Pct_0to3', 'Moth_HrsWorked_BefBirth', 
                        'Moth_HrsWorked_Avg_0to3', 'Moth_HrsWorked_0to1', 'Father_HH_0to3', 'GMom_0to3', 'MomCare', 
                        'RelCare', 'NonRelCare', 'Moth_Smoke_BefBirth', 'Alc_BefBirth', 'Breastfed', 
                        'Doctor_0to3', 'Dentist_0to3', 'Moth_WeightChange', 'Illness_1stYr', 'Premature', 
                        'Insurance_0to3', 'Medicaid_0to3']
mask = data['Sample90'] == 1
for covariate in covariates_to_impute:
    data[covariate + "_missing"] = np.nan
    data.loc[mask, covariate + "_missing"] = data.loc[mask, covariate].isna().astype(int)
    # TODO: check the above line
    data_temp = data.loc[mask].copy()
    data_temp[covariate] = data_temp[covariate].replace(-np.inf, np.finfo(np.float64).min)
    conditional_means = data_temp.groupby(['Black', 'Hispanic', 'male'])[covariate].transform('mean')
    data.loc[mask, covariate] = data.loc[mask, covariate].fillna(conditional_means)

# only consider the sample
data = data.loc[mask]
data.reset_index(drop=True, inplace=True)
print(len(data)) # 1455

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/600107391.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[covariate + "_missing"] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/600107391.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[covariate + "_missing"] = np.nan
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/600107391.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

1455


In [11]:
# Longitudinal data
for year in range(86, 106, 2):
    data[f'AgeTest_Yr{year}'] = data[f'PPVTAge{year}'] // 12
    data[f'Test_Pct{year}'] = data[[f'PPVT_Pct{year}', f'PIATMT_Pct{year}', f'PIATRR_Pct{year}']].mean(axis=1)

# Preserve the data
data = pd.wide_to_long(data, stubnames=['AgeTest_Yr', 'Test_Pct', 'PPVT_Raw', 'PIATMT_Raw', 'PIATRR_Raw', 'PPVT_Pct', 'PIATMT_Pct', 'PIATRR_Pct', 'BPI_Pct', 'BPIAS_Pct'], i=['ChildID'], j='Year', sep='')
mask = (data["AgeTest_Yr"] >= 5) & (data["AgeTest_Yr"] <= 14)
data = data.loc[mask]
data.reset_index(drop=False, inplace=True)
data["Group_5to6"] = ((5 <= data["AgeTest_Yr"]) & (data["AgeTest_Yr"] <= 6)).astype(int)
data["Group_7to10"] = ((7 <= data["AgeTest_Yr"]) & (data["AgeTest_Yr"] <= 10)).astype(int)
data["Group_11to14"] = ((11 <= data["AgeTest_Yr"]) & (data["AgeTest_Yr"] <= 14)).astype(int)

for grade in ["5to6", "7to10", "11to14"]:
    mask = data[f"Group_{grade}"] == 1
    for test in ["PIATMT", "PIATRR", "PPVT", "BPI", "BPIAS"]:
        data[f'{test}_std_{grade}'] = np.nan
        data.loc[mask, f'{test}_std_{grade}'] = (data.loc[mask, f'{test}_Pct'] - data.loc[mask, f'{test}_Pct'].mean()) / data.loc[mask, f'{test}_Pct'].std()
    data[f'Test_std_{grade}'] = np.nan
    data.loc[mask, f'Test_std_{grade}'] = data.loc[mask, [f'{t}_std_{grade}' for t in ["PIATMT", "PIATRR", "PPVT"]]].mean(axis=1)
    data.loc[mask, f'Test_std_{grade}'] = (data.loc[mask, f'Test_std_{grade}'] - data.loc[mask, f'Test_std_{grade}'].mean()) / data.loc[mask, f'Test_std_{grade}'].std()

for test in ["PIATMT", "PIATRR", "PPVT", "BPI", "BPIAS", "Test"]:
    data[f'{test}_std'] = np.nan
    data.loc[data["Group_5to6"] == 1, f'{test}_std'] = data.loc[data["Group_5to6"] == 1, f'{test}_std_5to6']
    data.loc[data["Group_7to10"] == 1, f'{test}_std'] = data.loc[data["Group_7to10"] == 1, f'{test}_std_7to10']
    data.loc[data["Group_11to14"] == 1, f'{test}_std'] = data.loc[data["Group_11to14"] == 1, f'{test}_std_11to14']

data["non_male"] = 1 - data["male"]
data.rename(columns={"male": "Male", "non_male": "NonMale"}, inplace=True)

data["lowAFQT"] = (data["impAFQT_std"] <= -1).astype(int)
data.loc[data["impAFQT_std"].isna(), "lowAFQT"] = np.nan
data['NonlowAFQT'] = 1-data['lowAFQT']

for grade in ["5to6", "7to10", "11to14"]:
    data[f'HS_{grade}'] = 0
    mask = (data[f'Group_{grade}'] == 1) & (data['headstart_fixedeffect_indicator_90'] == 1)
    data.loc[mask, f'HS_{grade}'] = 1
    mask = data['headstart_fixedeffect_indicator_90'].isna()
    data.loc[mask, f'HS_{grade}'] = np.nan

    data[f"Pre_{grade}"] = 0
    mask = (data[f'Group_{grade}'] == 1) & (data['preschool_fixedeffect_indicator_90'] == 1)
    data.loc[mask, f"Pre_{grade}"] = 1
    mask = data['preschool_fixedeffect_indicator_90'].isna()
    data.loc[mask, f"Pre_{grade}"] = np.nan

for group in ["Male", "NonMale", "Black", "NonBlack", "lowAFQT", "NonlowAFQT"]:
    data[f'HS_{group}'] = np.nan
    mask = (data['headstart_fixedeffect_indicator_90'] == 1) & (data[group] == 1)
    data.loc[mask, f'HS_{group}'] = 1
    mask = (data[f'HS_{group}'] != 1) & (data['headstart_fixedeffect_indicator_90'].notna())
    data.loc[mask, f'HS_{group}'] = 0

    data[f'Pre_{group}'] = np.nan
    mask = (data['preschool_fixedeffect_indicator_90'] == 1) & (data[group] == 1)
    data.loc[mask, f'Pre_{group}'] = 1
    mask = (data[f'Pre_{group}'] != 1) & (data['preschool_fixedeffect_indicator_90'].notna())
    data.loc[mask, f'Pre_{group}'] = 0

for grade in ["5to6", "7to10", "11to14"]:
    for group in ["Male", "NonMale", "Black", "NonBlack", "lowAFQT", "NonlowAFQT"]:
        data[f'HS_{group}_{grade}'] = np.nan
        mask = (data[f'Group_{grade}'] == 1) & (data['headstart_fixedeffect_indicator_90'] == 1) & (data[group] == 1)
        data.loc[mask, f'HS_{group}_{grade}'] = 1
        mask = (data[f'HS_{group}_{grade}'] != 1) & (data['headstart_fixedeffect_indicator_90'].notna())
        data.loc[mask, f'HS_{group}_{grade}'] = 0

        data[f'Pre_{group}_{grade}'] = np.nan
        mask = (data[f'Group_{grade}'] == 1) & (data['preschool_fixedeffect_indicator_90'] == 1) & (data[group] == 1)
        data.loc[mask, f'Pre_{group}_{grade}'] = 1
        mask = (data[f'Pre_{group}_{grade}'] != 1) & (data['preschool_fixedeffect_indicator_90'].notna())
        data.loc[mask, f'Pre_{group}_{grade}'] = 0

data["Group_5to14"] = 1

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3790053760.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'AgeTest_Yr{year}'] = data[f'PPVTAge{year}'] // 12
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3790053760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'AgeTest_Yr{year}'] = data[f'PPVTAge{year}'] // 12
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3790053760.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [12]:
# Table 3
column_names = [str(i) for i in range(1, 6)]
_row_names = ["Head Start 5to6", "Head Start 7to10", "Head Start 11to14", "Other Preschool 5to6", "Other Preschool 7to10", "Other Preschool 11to14"]
_row_names += ["Permanent income (standardized)", "Maternal AFQT (standardized)", "Mom high school", "Mom some college"]
row_names = []
for row_name in _row_names:
    row_names.append(row_name)
    row_names.append(row_name + " se/std")
    row_names.append(row_name + " p-value")
row_names += ["p (all age effects equal—Head Start)", "Pre-treatment covariates", "Sibling fixed effects", "R2", "Sample size"]

table3 = pd.DataFrame(index=row_names, columns=column_names)


def _generate_column_default(table3, model, idx):
    table3.loc["Head Start 5to6", str(idx)] = model.params["HS_5to6"]
    table3.loc["Head Start 5to6 se/std", str(idx)] = model.bse["HS_5to6"]
    table3.loc["Head Start 5to6 p-value", str(idx)] = model.t_test("HS_5to6 = 0").pvalue
    table3.loc["Head Start 7to10", str(idx)] = model.params["HS_7to10"]
    table3.loc["Head Start 7to10 se/std", str(idx)] = model.bse["HS_7to10"]
    table3.loc["Head Start 7to10 p-value", str(idx)] = model.t_test("HS_7to10 = 0").pvalue
    table3.loc["Head Start 11to14", str(idx)] = model.params["HS_11to14"]
    table3.loc["Head Start 11to14 se/std", str(idx)] = model.bse["HS_11to14"]
    table3.loc["Head Start 11to14 p-value", str(idx)] = model.t_test("HS_11to14 = 0").pvalue
    table3.loc["Other Preschool 5to6", str(idx)] = model.params["Pre_5to6"]
    table3.loc["Other Preschool 5to6 se/std", str(idx)] = model.bse["Pre_5to6"]
    table3.loc["Other Preschool 5to6 p-value", str(idx)] = model.t_test("Pre_5to6 = 0").pvalue
    table3.loc["Other Preschool 7to10", str(idx)] = model.params["Pre_7to10"]
    table3.loc["Other Preschool 7to10 se/std", str(idx)] = model.bse["Pre_7to10"]
    table3.loc["Other Preschool 7to10 p-value", str(idx)] = model.t_test("Pre_7to10 = 0").pvalue
    table3.loc["Other Preschool 11to14", str(idx)] = model.params["Pre_11to14"]
    table3.loc["Other Preschool 11to14 se/std", str(idx)] = model.bse["Pre_11to14"]
    table3.loc["Other Preschool 11to14 p-value", str(idx)] = model.t_test("Pre_11to14 = 0").pvalue
    table3.loc["p (all age effects equal—Head Start)", str(idx)] = model.f_test("HS_5to6 = HS_7to10 = HS_11to14").pvalue
    table3.loc["R2", str(idx)] = model.rsquared
    table3.loc["Sample size", str(idx)] = 1455 # TODO: hardcoded value


mask = data['Test_std'].notna()
baseline_covariates = ["HS_5to6", "HS_7to10", "HS_11to14", "Pre_5to6", "Pre_7to10", "Pre_11to14", "Male", "C(AgeTest_Yr)", "C(Year)"]
standard_covariates = covariates_to_impute + [cov + "_missing" for cov in covariates_to_impute]
additional_covariates = ["permanent_family_income_std", "impAFQT_std", "mother_highschool", "mother_somecollege"]
family_fixed_effects = ["C(MotherID)"]
# NOTE: in the paper, the author claims to adjust for first_born status, but this is not the case in the code

# column 1
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
_generate_column_default(table3, model, 1)
table3.loc["Pre-treatment covariates", "1"] = False
table3.loc["Sibling fixed effects", "1"] = False

# column 2
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates + standard_covariates), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
_generate_column_default(table3, model, 2)
table3.loc["Pre-treatment covariates", "2"] = True
table3.loc["Sibling fixed effects", "2"] = False

# column 3
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates + standard_covariates + additional_covariates), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
_generate_column_default(table3, model, 3)
table3.loc["Pre-treatment covariates", "3"] = True
table3.loc["Sibling fixed effects", "3"] = False
table3.loc["Permanent income (standardized)", "3"] = model.params["permanent_family_income_std"]
table3.loc["Permanent income (standardized) se/std", "3"] = model.bse["permanent_family_income_std"]
table3.loc["Permanent income (standardized) p-value", "3"] = model.t_test("permanent_family_income_std = 0").pvalue
table3.loc["Maternal AFQT (standardized)", "3"] = model.params["impAFQT_std"]
table3.loc["Maternal AFQT (standardized) se/std", "3"] = model.bse["impAFQT_std"]
table3.loc["Maternal AFQT (standardized) p-value", "3"] = model.t_test("impAFQT_std = 0").pvalue
table3.loc["Mom high school", "3"] = model.params["mother_highschool"]
table3.loc["Mom high school se/std", "3"] = model.bse["mother_highschool"]
table3.loc["Mom high school p-value", "3"] = model.t_test("mother_highschool = 0").pvalue
table3.loc["Mom some college", "3"] = model.params["mother_somecollege"]
table3.loc["Mom some college se/std", "3"] = model.bse["mother_somecollege"]
table3.loc["Mom some college p-value", "3"] = model.t_test("mother_somecollege = 0").pvalue

# column 4
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
_generate_column_default(table3, model, 4)
table3.loc["Pre-treatment covariates", "4"] = False
table3.loc["Sibling fixed effects", "4"] = True

# column 5
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
_generate_column_default(table3, model, 5)
table3.loc["Pre-treatment covariates", "5"] = True
table3.loc["Sibling fixed effects", "5"] = True

table3.to_csv("table3.csv")
table3

,1,2,3,4,5
Head Start 5to6,-0.062599,0.060392,0.067084,0.130202,0.146343
Head Start 5to6 se/std,0.085362,0.078521,0.075246,0.090472,0.089245
Head Start 5to6 p-value,0.46335288380337414,0.4418248532361223,0.37264916470467,0.15011289240066253,0.10104822728752048
Head Start 7to10,-0.148909,0.004322,0.034212,0.113062,0.127851
Head Start 7to10 se/std,0.068054,0.062423,0.059222,0.063253,0.063647
Head Start 7to10 p-value,0.028662982358712192,0.9448021824111316,0.5634659866932015,0.07386212095851483,0.04456165774594422
Head Start 11to14,-0.23041,-0.080992,-0.048434,0.032131,0.050266
Head Start 11to14 se/std,0.066189,0.060988,0.058485,0.064496,0.065064
Head Start 11to14 p-value,0.0004993915975453988,0.18417763863376824,0.40758727897899394,0.6183491832691793,0.43978184138694987
Other Preschool 5to6,0.295608,0.085168,0.040384,-0.062856,-0.034753


In [13]:
# nonscore, 5-14
# Repeat grade
data.loc[data["Repeat92"] == 6, "Repeat92"] = np.nan
data["Repeat94"] = data[["Repeat_K94"] + [f"Repeat_{94 + 100*i}" for i in range(1,9)]].max(axis=1)
data["Repeat96"] = data[["Repeat_K96"] + [f"Repeat_{96 + 100*i}" for i in range(1,13)]].max(axis=1)
data.loc[data["Repeat_None94"] == 1, "Repeat94"] = 0
data.loc[data["Repeat_None96"] == 1, "Repeat96"] = 0
data.rename(columns={"RepeatNone98":"Repeat_None98"}, inplace=True)
del data["Repeat_YA98"]
data["Repeat98"] = data[["Repeat_K98"] + [f"Repeat_{98 + 100*i}" for i in range(1,9)]].max(axis=1)
data.loc[data["Repeat_None98"] == 1, "Repeat98"] = 0

data["Repeat"] = data[[f"Repeat{yr}" for yr in range(88, 106, 2)]].max(axis=1)

# Learning Disability
for year in range(86, 102, 2):
	data[f'tempLD{year}'] = data[f'LD{year}']
	mask = (data[f'HealthCond{year}'].notna()) & (data[f'tempLD{year}'] != 1)
	data.loc[mask, f'tempLD{year}'] = 0
data["LD"] = data[[f'tempLD{year}' for year in range(86, 102, 2)]].max(axis=1)

data["LD_before"] = np.nan
for year in range(86, 102, 2):
    mask = (data[f'tempLD{year}'] == 1) & (data[f'cleaned_age_yr_{year}'] < 5)
    data.loc[mask, "LD_before"] = 1

data.loc[data["LD_before"] == 1, "LD"] = np.nan
data.drop(columns=[f'tempLD{year}' for year in range(86, 102, 2)], inplace=True)
data.drop(columns=[col for col in data.columns if col.startswith('temp')], inplace=True)



# Long term
# HS Grad
data['HSGrad'] = ((data['YA_Educ104'] >= 12) & (data['YA_Educ104'].notna())).astype(float)
data.loc[data['YA_Educ104'].isna(), 'HSGrad'] = np.nan
data['GED'] = data[[col for col in data.columns if col.startswith('GED')]].max(axis=1)
data['HSGrad_GED'] = data['HSGrad']
data.loc[(data['HSGrad_GED'] == 1) & (data['GED'] == 2), 'HSGrad_GED'] = np.nan

# College
data['HighGradeAtt'] = data[[col for col in data.columns if col.startswith('HighGrade_Att')]].max(axis=1)
data['someCollAtt'] = ((data['YA_Educ104'] >= 13) & (data['YA_Educ104'].notna())).astype(int)
data.loc[data['YA_Educ104'].isna(), 'someCollAtt'] = np.nan
data.loc[(data['HighGradeAtt'] > 12) & (data['HighGradeAtt'].notna()), 'someCollAtt'] = 1

# Idleness
data['Wages'] = data['Wages104']
data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 2002), 'Wages'] = data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 2002), 'Wages102']
data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 2000), 'Wages'] = data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 2000), 'Wages100']
data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1998), 'Wages'] = data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1998), 'Wages98']
data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1996), 'Wages'] = data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1996), 'Wages96']
data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1994), 'Wages'] = data.loc[(data['Wages'].isna()) & (data['YA_LastInterview'] == 1994), 'Wages94']
for x in [104, 102, 100, 98, 96, 94]:
	data[f'PosWages{x}'] = (data[f'Wages{x}'] > 0) & (data[f'Wages{x}'].notna())
	data.loc[data[f'Wages{x}'].isna(), f'PosWages{x}'] = np.nan
data['PosWages'] = data['PosWages104']
data.loc[data['PosWages104'].isna() & (data['Wages_Est104'] == 1), 'PosWages'] = 0
data.loc[data['PosWages104'].isna() & data['Wages_Est104'].notna() & (data['Wages_Est104'] > 1), 'PosWages'] = 1
data.loc[data['PosWages'].isna() & (data['YA_LastInterview'] == 2002), 'PosWages'] = data.loc[data['PosWages'].isna() & (data['YA_LastInterview'] == 2002), 'PosWages102']
data.loc[data['PosWages'].isna() & data['PosWages102'].isna() & (data['Wages_Est102'] == 1) & (data['YA_LastInterview'] == 2002), 'PosWages'] = 0
data.loc[data['PosWages'].isna() & data['PosWages102'].isna() & data['Wages_Est102'].notna() & (data['Wages_Est102'] > 1) & (data['YA_LastInterview'] == 2002), 'PosWages'] = 1
data.loc[data['PosWages'].isna() & (data['YA_LastInterview'] == 2000), 'PosWages'] = data.loc[data['PosWages'].isna() & (data['YA_LastInterview'] == 2000), 'PosWages100']
data.loc[data['PosWages'].isna() & data['PosWages100'].isna() & (data['Wages_Est100'] == 1) & (data['YA_LastInterview'] == 2000), 'PosWages'] = 0
data.loc[data['PosWages'].isna() & data['PosWages100'].isna() & data['Wages_Est100'].notna() & (data['Wages_Est100'] > 1) & (data['YA_LastInterview'] == 2000), 'PosWages'] = 1

data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1998), 'PosWages'] = data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1998), 'PosWages98']
data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1996), 'PosWages'] = data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1996), 'PosWages96']
data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1994), 'PosWages'] = data.loc[(data['PosWages'].isna()) & (data['YA_LastInterview'] == 1994), 'PosWages94']

data['InSchool'] = data['InSchool104']
data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 2002), 'InSchool'] = data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 2002), 'InSchool102']
data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 2000), 'InSchool'] = data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 2000), 'InSchool100']
data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1998), 'InSchool'] = data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1998), 'InSchool98']
data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1996), 'InSchool'] = data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1996), 'InSchool96']
data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1994), 'InSchool'] = data.loc[(data['InSchool'].isna()) & (data['YA_LastInterview'] == 1994), 'InSchool94']

data['Idle'] = np.nan
data.loc[data["InSchool"].notna() | data["PosWages"].notna(), "Idle"] = 0
data.loc[(data["InSchool"] == 0) & (data["PosWages"] == 0), "Idle"] = 1
data.loc[(data["InSchool"].isna()) & (data["PosWages"] == 0), "Idle"] = 1
data.loc[(data["InSchool"] == 0) & (data["PosWages"].isna()), "Idle"] = 1

# Crime
data["Convicted"] = data[[col for col in data.columns if col.startswith('Convicted')]].max(axis=1)
data["Probation"] = data[[col for col in data.columns if col.startswith('Probation')]].max(axis=1)
data["Sentenced"] = data[[col for col in data.columns if col.startswith('Sentenced')]].max(axis=1)
for x in [94, 96, 98, 100, 102, 104]:
	data[f'Prison{x}'] = np.nan
	data.loc[data[f'Resid{x}'] == 5, f'Prison{x}'] = 1
	data.loc[(data[f'Resid{x}'].notna()) & (data[f'Prison{x}'] != 1), f'Prison{x}'] = 0
data["Prison"] = data[[f'Prison{x}' for x in [94, 96, 98, 100, 102, 104]]].max(axis=1)
data["Crime"] = data[["Convicted", "Probation", "Sentenced", "Prison"]].max(axis=1)

# Teen Parenthood
data["TeenPreg"] = np.nan
data.loc[data["Ageat1stBirth"] < 20, "TeenPreg"] = 1
data.loc[data["Ageat1stBirth"] >= 20, "TeenPreg"] = 0
data.loc[(data["TeenPreg"] != 1) & (data["YA_NumKids"].notna()), "TeenPreg"] = 0

# Poor Health
data['HealthReport'] = data[[col for col in data.columns if col.startswith('Health_Report')]].mean(axis=1)
data['PoorHealth'] = np.nan
data.loc[(data['HealthReport'] < 3) & (data['HealthReport'].notna()), 'PoorHealth'] = 1
data.loc[(data['HealthReport'] >= 3) & (data['HealthReport'].notna()), 'PoorHealth'] = 0

# dummy outcome
for outcome in ["LD", "Repeat", "HSGrad", "someCollAtt", "Idle", "Crime", "Prison", "TeenPreg", "PoorHealth", "HSGrad_GED"]:
	data[outcome + "_std"] = (data[outcome] - data[outcome].mean()) / data[outcome].std()
	data.loc[data[outcome].isna(), outcome + "_dummy"] = np.nan
for outcome in ["LD", "Repeat", "Idle", "Crime", "Prison", "TeenPreg", "PoorHealth"]:
	data[outcome + "_std"] *= -1
data["Sum_Adult"] = data[[outcome + "_std" for outcome in ["HSGrad", "someCollAtt", "Idle", "Crime", "TeenPreg", "PoorHealth"]]].sum(axis=1)
data["Sum_Adult"] = (data["Sum_Adult"] - data["Sum_Adult"].mean()) / data["Sum_Adult"].std()
data["Noncog"] = data[[outcome + "_std" for outcome in ["LD", "Repeat"]]].sum(axis=1)
data["Noncog"] = (data["Noncog"] - data["Noncog"].mean()) / data["Noncog"].std()

/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3151906628.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Repeat98"] = data[["Repeat_K98"] + [f"Repeat_{98 + 100*i}" for i in range(1,9)]].max(axis=1)
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3151906628.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Repeat"] = data[[f"Repeat{yr}" for yr in range(88, 106, 2)]].max(axis=1)
/var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_74938/3151906628.py:17: PerformanceWarn

In [14]:
# Table 5
column_names = ["All", "Black", "NonBlack", "Male", "NonMale", "lowAFQT", "NonlowAFQT"]
group_names = ["Repeat", "LD", "HSGrad", "HSGrad_GED", "someCollAtt", "Idle", "Crime", "TeenPreg", "PoorHealth"]
row_names = []
for group in group_names:
    row_names.append(group)
    row_names.append(group + " se/std")
    row_names.append(group + " p-value")
table5 = pd.DataFrame(index=row_names, columns=column_names)
covariates = lambda group: ["headstart_fixedeffect_indicator_90", "preschool_fixedeffect_indicator_90", "C(cleaned_age_yr_104)"] +\
            covariates_to_impute + [cov + "_missing" for cov in covariates_to_impute] + ["C(MotherID)"] + (["Male"] if group != "Male" else [])
group_covariates = lambda group: [f"HS_{group}", f"HS_Non{group}", f"Pre_{group}", f"Pre_Non{group}"] + covariates(group)[2:]
for outcome in group_names:
    mask = data[outcome].notna()
    model = smf.ols(f"{outcome} ~ " + " + ".join(covariates(group)), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table5.loc[outcome, "All"] = model.params["headstart_fixedeffect_indicator_90"]
    table5.loc[outcome + " se/std", "All"] = model.bse["headstart_fixedeffect_indicator_90"]
    table5.loc[outcome + " p-value", "All"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue

    for group in ["Black", "Male", "lowAFQT"]:
        model = smf.ols(f"{outcome} ~ " + " + ".join(group_covariates(group)), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
        table5.loc[outcome, group] = model.params[f"HS_{group}"]
        table5.loc[outcome + " se/std", group] = model.bse[f"HS_{group}"]
        table5.loc[outcome + " p-value", group] = model.t_test(f"HS_{group} = 0").pvalue
        table5.loc[outcome, f"Non{group}"] = model.params[f"HS_Non{group}"]
        table5.loc[outcome + " se/std", f"Non{group}"] = model.bse[f"HS_Non{group}"]
        table5.loc[outcome + " p-value", f"Non{group}"] = model.t_test(f"HS_Non{group} = 0").pvalue

table5.to_csv("table5.csv")
table5

,All,Black,NonBlack,Male,NonMale,lowAFQT,NonlowAFQT
Repeat,-0.041698,-0.065403,-0.015341,-0.084225,-0.00845,-0.12898,0.005466
Repeat se/std,0.051144,0.068219,0.076506,0.059283,0.063029,0.093781,0.061212
Repeat p-value,0.41489529812723724,0.3376997366690061,0.8410714196278483,0.1553956077086122,0.8933512073345268,0.16902693427746218,0.9288439050575381
LD,-0.060985,-0.065517,-0.057809,-0.030295,-0.090296,-0.107898,-0.035039
LD se/std,0.022764,0.029683,0.034735,0.030291,0.024513,0.044998,0.024772
LD p-value,0.007383711738846131,0.027298581253371427,0.09605673382390711,0.317251813235614,0.00022990626430757235,0.016493038611718577,0.15723407868416153
HSGrad,0.06645,0.099422,0.03086,0.046076,0.08657,0.164565,0.013432
HSGrad se/std,0.034212,0.044208,0.054354,0.044366,0.040335,0.059721,0.040955
HSGrad p-value,0.05209942445637254,0.02451328873927494,0.570199901458438,0.29901170052168835,0.03184993287182975,0.005859322238420654,0.7429401920792855
HSGrad_GED,0.052719,0.055641,0.050748,0.027466,0.076958,0.126895,0.009958


In [15]:
# Table 4
column_names = ["Test Scores, 5-6", "Test Scores, 7-10", "Test Scores, 11-14", "Test Scores, 5-14", 
                "Nontest score, 5-14", "Long term, 19+"]
row_names = ["Head Start, overall", "Head Start, overall, se", "Head Start, overall, p-value", "Other preschool, overall", "Other preschool, overall, se", "Other preschool, overall, p-value", "p (HS = preschool), overall",
             "Head Start, Black", "Head Start, Black, se", "Head Start, Black, p-value", "Head Start, NonBlack", "Head Start, NonBlack, se", "Head Start, NonBlack, p-value", "p (Black=NonBlack), HS",
             "Head Start, Male", "Head Start, Male, se", "Head Start, Male, p-value", "Head Start, NonMale", "Head Start, NonMale, se", "Head Start, NonMale, p-value", "p (Male=NonMale), HS",
             "Head Start, lowAFQT", "Head Start, lowAFQT, se", "Head Start, lowAFQT, p-value", "Head Start, NonlowAFQT", "Head Start, NonlowAFQT, se", "Head Start, NonlowAFQT, p-value", "p (lowAFQT=NonlowAFQT), HS"]
table4 = pd.DataFrame(index=row_names, columns=column_names)

baseline_covariates = ["HS_5to6", "HS_7to10", "HS_11to14", "Pre_5to6", "Pre_7to10", "Pre_11to14", "Male", "C(AgeTest_Yr)", "C(Year)"]
overall_baseline_covariates = ["headstart_fixedeffect_indicator_90", "preschool_fixedeffect_indicator_90", "Male", "C(AgeTest_Yr)", "C(Year)"]
standard_covariates = covariates_to_impute + [cov + "_missing" for cov in covariates_to_impute]
family_fixed_effects = ["C(MotherID)"]

# test scores
# overall
mask = data['Test_std'].notna()
model = smf.ols("Test_std ~ " + " + ".join(baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table4.loc["Head Start, overall", "Test Scores, 5-6"] = model.params["HS_5to6"]
table4.loc["Head Start, overall, se", "Test Scores, 5-6"] = model.bse["HS_5to6"]
table4.loc["Head Start, overall, p-value", "Test Scores, 5-6"] = model.t_test("HS_5to6 = 0").pvalue
table4.loc["Head Start, overall", "Test Scores, 7-10"] = model.params["HS_7to10"]
table4.loc["Head Start, overall, se", "Test Scores, 7-10"] = model.bse["HS_7to10"]
table4.loc["Head Start, overall, p-value", "Test Scores, 7-10"] = model.t_test("HS_7to10 = 0").pvalue
table4.loc["Head Start, overall", "Test Scores, 11-14"] = model.params["HS_11to14"]
table4.loc["Head Start, overall, se", "Test Scores, 11-14"] = model.bse["HS_11to14"]
table4.loc["Head Start, overall, p-value", "Test Scores, 11-14"] = model.t_test("HS_11to14 = 0").pvalue
table4.loc["Other preschool, overall", "Test Scores, 5-6"] = model.params["Pre_5to6"]
table4.loc["Other preschool, overall, se", "Test Scores, 5-6"] = model.bse["Pre_5to6"]
table4.loc["Other preschool, overall, p-value", "Test Scores, 5-6"] = model.t_test("Pre_5to6 = 0").pvalue
table4.loc["Other preschool, overall", "Test Scores, 7-10"] = model.params["Pre_7to10"]
table4.loc["Other preschool, overall, se", "Test Scores, 7-10"] = model.bse["Pre_7to10"]
table4.loc["Other preschool, overall, p-value", "Test Scores, 7-10"] = model.t_test("Pre_7to10 = 0").pvalue
table4.loc["Other preschool, overall", "Test Scores, 11-14"] = model.params["Pre_11to14"]
table4.loc["Other preschool, overall, se", "Test Scores, 11-14"] = model.bse["Pre_11to14"]
table4.loc["Other preschool, overall, p-value", "Test Scores, 11-14"] = model.t_test("Pre_11to14 = 0").pvalue

table4.loc["p (HS = preschool), overall", "Test Scores, 5-6"] = model.f_test("HS_5to6 = Pre_5to6").pvalue
table4.loc["p (HS = preschool), overall", "Test Scores, 7-10"] = model.f_test("HS_7to10 = Pre_7to10").pvalue
table4.loc["p (HS = preschool), overall", "Test Scores, 11-14"] = model.f_test("HS_11to14 = Pre_11to14").pvalue


model = smf.ols("Test_std ~ " + " + ".join(overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table4.loc["Head Start, overall", "Test Scores, 5-14"] = model.params["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, se", "Test Scores, 5-14"] = model.bse["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, p-value", "Test Scores, 5-14"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
table4.loc["Other preschool, overall", "Test Scores, 5-14"] = model.params["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, se", "Test Scores, 5-14"] = model.bse["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, p-value", "Test Scores, 5-14"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue

table4.loc["p (HS = preschool), overall", "Test Scores, 5-14"] = model.f_test("headstart_fixedeffect_indicator_90 = preschool_fixedeffect_indicator_90").pvalue

# by subgroup
for name in ["Male", "Black", "lowAFQT"]:
    _baseline_covariates = [f"HS_{name}_5to6", f"HS_{name}_7to10", f"HS_{name}_11to14", 
                            f"Pre_{name}_5to6", f"Pre_{name}_7to10", f"Pre_{name}_11to14",
                            f"HS_Non{name}_5to6", f"HS_Non{name}_7to10", f"HS_Non{name}_11to14",
                            f"Pre_Non{name}_5to6", f"Pre_Non{name}_7to10", f"Pre_Non{name}_11to14"]
    _baseline_covariates += ["C(AgeTest_Yr)", "C(Year)"]
    if name != "Male":
        _baseline_covariates += ["Male"]
    model = smf.ols("Test_std ~ " + " + ".join(_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table4.loc[f"Head Start, {name}", "Test Scores, 5-6"] = model.params[f"HS_{name}_5to6"]
    table4.loc[f"Head Start, {name}, se", "Test Scores, 5-6"] = model.bse[f"HS_{name}_5to6"]
    table4.loc[f"Head Start, {name}, p-value", "Test Scores, 5-6"] = model.t_test(f"HS_{name}_5to6 = 0").pvalue
    table4.loc[f"Head Start, {name}", "Test Scores, 7-10"] = model.params[f"HS_{name}_7to10"]
    table4.loc[f"Head Start, {name}, se", "Test Scores, 7-10"] = model.bse[f"HS_{name}_7to10"]
    table4.loc[f"Head Start, {name}, p-value", "Test Scores, 7-10"] = model.t_test(f"HS_{name}_7to10 = 0").pvalue
    table4.loc[f"Head Start, {name}", "Test Scores, 11-14"] = model.params[f"HS_{name}_11to14"]
    table4.loc[f"Head Start, {name}, se", "Test Scores, 11-14"] = model.bse[f"HS_{name}_11to14"]
    table4.loc[f"Head Start, {name}, p-value", "Test Scores, 11-14"] = model.t_test(f"HS_{name}_11to14 = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Test Scores, 5-6"] = model.params[f"HS_Non{name}_5to6"]
    table4.loc[f"Head Start, Non{name}, se", "Test Scores, 5-6"] = model.bse[f"HS_Non{name}_5to6"]
    table4.loc[f"Head Start, Non{name}, p-value", "Test Scores, 5-6"] = model.t_test(f"HS_Non{name}_5to6 = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Test Scores, 7-10"] = model.params[f"HS_Non{name}_7to10"]
    table4.loc[f"Head Start, Non{name}, se", "Test Scores, 7-10"] = model.bse[f"HS_Non{name}_7to10"]
    table4.loc[f"Head Start, Non{name}, p-value", "Test Scores, 7-10"] = model.t_test(f"HS_Non{name}_7to10 = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Test Scores, 11-14"] = model.params[f"HS_Non{name}_11to14"]
    table4.loc[f"Head Start, Non{name}, se", "Test Scores, 11-14"] = model.bse[f"HS_Non{name}_11to14"]
    table4.loc[f"Head Start, Non{name}, p-value", "Test Scores, 11-14"] = model.t_test(f"HS_Non{name}_11to14 = 0").pvalue

    table4.loc[f"p ({name}=Non{name}), HS", "Test Scores, 5-6"] = model.f_test(f"HS_{name}_5to6 = HS_Non{name}_5to6").pvalue
    table4.loc[f"p ({name}=Non{name}), HS", "Test Scores, 7-10"] = model.f_test(f"HS_{name}_7to10 = HS_Non{name}_7to10").pvalue
    table4.loc[f"p ({name}=Non{name}), HS", "Test Scores, 11-14"] = model.f_test(f"HS_{name}_11to14 = HS_Non{name}_11to14").pvalue


    _overall_baseline_covariates = [f"HS_{name}",
                                    f"Pre_{name}",
                                    f"HS_Non{name}",
                                    f"Pre_Non{name}",]
    _overall_baseline_covariates += ["C(AgeTest_Yr)", "C(Year)"]
    if name != "Male":
        _overall_baseline_covariates += ["Male"]
    model = smf.ols("Test_std ~ " + " + ".join(_overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table4.loc[f"Head Start, {name}", "Test Scores, 5-14"] = model.params[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, se", "Test Scores, 5-14"] = model.bse[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, p-value", "Test Scores, 5-14"] = model.t_test(f"HS_{name} = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Test Scores, 5-14"] = model.params[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, se", "Test Scores, 5-14"] = model.bse[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, p-value", "Test Scores, 5-14"] = model.t_test(f"HS_Non{name} = 0").pvalue

    table4.loc[f"p ({name}=Non{name}), HS", "Test Scores, 5-14"] = model.f_test(f"HS_{name} = HS_Non{name}").pvalue


# noncognitive outcomes, 5-14
mask = data['Noncog'].notna()
model = smf.ols("Noncog ~ " + " + ".join(overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table4.loc["Head Start, overall", "Nontest score, 5-14"] = model.params["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, se", "Nontest score, 5-14"] = model.bse["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, p-value", "Nontest score, 5-14"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
table4.loc["Other preschool, overall", "Nontest score, 5-14"] = model.params["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, se", "Nontest score, 5-14"] = model.bse["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, p-value", "Nontest score, 5-14"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue

table4.loc["p (HS = preschool), overall", "Nontest score, 5-14"] = model.f_test("headstart_fixedeffect_indicator_90 = preschool_fixedeffect_indicator_90").pvalue

for name in ["Male", "Black", "lowAFQT"]:
    _overall_baseline_covariates = [f"HS_{name}",
                                    f"Pre_{name}",
                                    f"HS_Non{name}",
                                    f"Pre_Non{name}",]
    _overall_baseline_covariates += ["C(AgeTest_Yr)", "C(Year)"]
    if name != "Male":
        _overall_baseline_covariates += ["Male"]
    model = smf.ols("Noncog ~ " + " + ".join(_overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table4.loc[f"Head Start, {name}", "Nontest score, 5-14"] = model.params[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, se", "Nontest score, 5-14"] = model.bse[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, p-value", "Nontest score, 5-14"] = model.t_test(f"HS_{name} = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Nontest score, 5-14"] = model.params[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, se", "Nontest score, 5-14"] = model.bse[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, p-value", "Nontest score, 5-14"] = model.t_test(f"HS_Non{name} = 0").pvalue

    table4.loc[f"p ({name}=Non{name}), HS", "Nontest score, 5-14"] = model.f_test(f"HS_{name} = HS_Non{name}").pvalue

# long term outcomes
mask = data['Sum_Adult'].notna()
model = smf.ols("Sum_Adult ~ " + " + ".join(overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
table4.loc["Head Start, overall", "Long term, 19+"] = model.params["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, se", "Long term, 19+"] = model.bse["headstart_fixedeffect_indicator_90"]
table4.loc["Head Start, overall, p-value", "Long term, 19+"] = model.t_test("headstart_fixedeffect_indicator_90 = 0").pvalue
table4.loc["Other preschool, overall", "Long term, 19+"] = model.params["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, se", "Long term, 19+"] = model.bse["preschool_fixedeffect_indicator_90"]
table4.loc["Other preschool, overall, p-value", "Long term, 19+"] = model.t_test("preschool_fixedeffect_indicator_90 = 0").pvalue

table4.loc["p (HS = preschool), overall", "Long term, 19+"] = model.f_test("headstart_fixedeffect_indicator_90 = preschool_fixedeffect_indicator_90").pvalue

for name in ["Male", "Black", "lowAFQT"]:
    _overall_baseline_covariates = [f"HS_{name}",
                                    f"Pre_{name}",
                                    f"HS_Non{name}",
                                    f"Pre_Non{name}",]
    _overall_baseline_covariates += ["C(AgeTest_Yr)", "C(Year)"]
    if name != "Male":
        _overall_baseline_covariates += ["Male"]
    model = smf.ols("Sum_Adult ~ " + " + ".join(_overall_baseline_covariates + standard_covariates + family_fixed_effects), data=data.loc[mask]).fit(cov_type='cluster', cov_kwds={'groups': data.loc[mask, 'MotherID']})
    table4.loc[f"Head Start, {name}", "Long term, 19+"] = model.params[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, se", "Long term, 19+"] = model.bse[f"HS_{name}"]
    table4.loc[f"Head Start, {name}, p-value", "Long term, 19+"] = model.t_test(f"HS_{name} = 0").pvalue
    table4.loc[f"Head Start, Non{name}", "Long term, 19+"] = model.params[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, se", "Long term, 19+"] = model.bse[f"HS_Non{name}"]
    table4.loc[f"Head Start, Non{name}, p-value", "Long term, 19+"] = model.t_test(f"HS_Non{name} = 0").pvalue

    table4.loc[f"p ({name}=Non{name}), HS", "Long term, 19+"] = model.f_test(f"HS_{name} = HS_Non{name}").pvalue

table4.to_csv("table4.csv")
table4


,"Test Scores, 5-6","Test Scores, 7-10","Test Scores, 11-14","Test Scores, 5-14","Nontest score, 5-14","Long term, 19+"
"Head Start, overall",0.146343,0.127851,0.050266,0.096006,0.20832,0.237244
"Head Start, overall, se",0.089245,0.063647,0.065064,0.060271,0.089733,0.081203
"Head Start, overall, p-value",0.10104822728752048,0.04456165774594422,0.43978184138694987,0.11118319114094476,0.020256867249719545,0.003482059284241549
"Other preschool, overall",-0.034753,0.033746,-0.025147,-0.002532,0.095127,0.072341
"Other preschool, overall, se",0.086667,0.066156,0.071551,0.063547,0.097689,0.077079
"Other preschool, overall, p-value",0.688422618589044,0.6099846080937569,0.7252478089046317,0.968213242584602,0.33017179422968834,0.3479684085469896
"p (HS = preschool), overall",0.061413,0.228421,0.342406,0.189426,0.320812,0.10198
"Head Start, Black",0.251355,0.143366,-0.00146,0.095295,0.239401,0.272902
"Head Start, Black, se",0.10908,0.079076,0.078149,0.075258,0.121528,0.114016
"Head Start, Black, p-value",0.021205074038067567,0.06982915895486999,0.9850912790580943,0.20542536709141146,0.04884709668394021,0.016686215494564737


In [16]:
for res in ["Test_std", "Noncog", "Sum_Adult"]:
    mask = data[res].notna()
    table_appendix.loc[res, "Mean"] = data.loc[mask, res].mean()
    table_appendix.loc[res, "Std Dev"] = data.loc[mask, res].std()
    table_appendix.loc[res, "Median"] = data.loc[mask, res].median()
    mask = (data["headstart_fixedeffect_indicator_90"] == 1) & data[res].notna()
    table_appendix.loc[res, "Head Start Mean"] = data.loc[mask, res].mean()
    mask = (data["preschool_fixedeffect_indicator_90"] == 1) & data[res].notna()
    table_appendix.loc[res, "Other Preschool Mean"] = data.loc[mask, res].mean()
    mask = (data["headstart_fixedeffect_indicator_90"] == 0) & (data["preschool_fixedeffect_indicator_90"] == 0) & data[res].notna()
    table_appendix.loc[res, "No Preschool Mean"] = data.loc[mask, res].mean()

table_appendix.to_csv("table_appendix.csv")
table_appendix

,Mean,Median,Std Dev,Head Start Mean,Other Preschool Mean,No Preschool Mean,Total Sample Size
Test_std,-0.0,-0.046595,0.999819,-0.209729,0.228275,-0.043883,NaN
Noncog,0.0,0.619999,0.999995,-0.059064,0.097838,-0.049033,NaN
Sum_Adult,-0.0,0.162315,1.0,-0.118317,0.212288,-0.115814,NaN
male,0.490722,0.0,0.500086,0.493617,0.488844,0.489837,1455
reside_with_mother_0to3,0.688584,1.0,0.46324,0.688636,0.708333,0.668103,1384
prexisting_health_conditions,0.041908,0.0,0.20045,0.045455,0.0375,0.043103,1384
very_low_birth_weight,0.016854,0.0,0.128769,0.017505,0.010288,0.022869,1424
logBW,4.710516,4.744932,0.253003,4.698948,4.729973,4.701848,1424
log_income_0to3,10.046628,10.038945,0.728076,9.813677,10.302084,10.006404,1386
log_income_at_3,9.996653,10.023771,0.90249,9.708326,10.274472,9.988104,1160
